This notebook has been created to streamline all the tests we have definined for our implementations, be it to assert their intended funcionality or to challenge them against new situations. This document is divided into various sections, roughly following the logic of collecting existing basics to eventually recombine them to something original. Each test will have a brief explanation and a discussion of any results that are worth noting.

Note that this document contains everything required to interface with our full corpus of code. Feel free to modify things to make it perform different tasks.
# Prerequisites

To run these tests, execute the following lines of code once. Then, each test can be run standalone. Unless noted otherwise, all algorithms are implementations of existing research which is referenced in the readme.

First, a few imports we will need in most tests. The point of reference is the root folder of this project, which is one step above.

In [ ]:
import sys
sys.path.append('../')

import helpers.masterTester as tester
from environment.Gaussian_noise import Gaussian_noise as gn
import numpy as np

The testcode will refer to some global variables that we can set her, outside the original file. (If not set, they will be defaulted to some meaningful values.)

For best performance, set AVAILABLE_CORES to the number of cores on your machine or lower. High values may cause excessive memory usage and crashes. Set quiet to False to see information on process assignment.

In [ ]:
tester.ERRORBAR_COUNT = 40
tester.AVAILABLE_CORES = 8
tester.QUIET = True

Generally, our results are written into files, then read in again and plotted. The test function streamlines this process, but you can still call the internal functions directly, for example to plot something again without recalculating it, or to add more subtests to existing data, which will then all be plotted together. Just be aware that the test function will delete previous data on default before calling testOnly, readAllResults and plotResults.

In [ ]:
tester.readAllResults()
tester.plotResults()

**Important:** Some of the tests will take very long to execute on an average system. When running them locally, you should put the shell from which you launched jupyter notebook into focus. Otherwise (at least in Windows 11), the scheduler of you OS will assign less processing power to the test, making it take even longer. If you are still not satisfied, you can try to copy the code above and the test code into a .py file and execute it normally.

# Stationary, Single Objective
These cases will test the implementations of strategies that operate on the standard MAB-model. Each arm has a fixed expectation value unknown to the agent and will return this value plus random noise as feedback when chosen. The agent will accumulate regret whenever it does not choose the best arm, so it needs to find it quickly while being sufficiently certain that no other arm is better on average.

Note that many of these tests are from an early phase of our research, which is why they do not make use of the streamlined interfaces that we utilize later on.

## Test 1
This first test will run the well known strategies. These include different flavours of UCB, which works by estimating by how much an arm's average might still improve as long as there is insufficient data on that arm and adds that onto the average when choosing the current best arm. The other strategy is Epsilon Greedy, which simply picks either the arm with the best raw average or some random arm, but gradually reduces the amount of random choices.

In [ ]:
# -*- coding: utf-8 -*-
"""
Created on Fri May 31 11:17:30 2024

@author: Xiaotong

test the algorithms in paper "Finite-time Analysis of the Multiarmed Bandit Problem"
"""

from environment.Gaussian_noise import Gaussian_noise as gn
from environment.env import env_stochastic
from algorithms.UCB2 import UCB2
from algorithms.epsilion import EpsilonGreedy
from algorithms.UCB1 import UCB1
from algorithms.UCB1N import UCB1N 

import numpy as np
import matplotlib.pyplot as plt

num_arm = 5
T = 100

mu = np.array([0.1, 0.9, 0.4, 0.3, 0.6])
Trial = 10
noise = gn(1,0,0.01,[-0.1,0.1])
env = env_stochastic(num_arm, mu, noise)


algorithms = [UCB2(T, num_arm, alpha=0.1), EpsilonGreedy(T, num_arm, c=0.01, d=0.05), UCB1(T, num_arm), UCB1N(T, num_arm)]
algorithm_names = ['UCB2', 'Epsilon-Greedy', 'UCB1', 'UCB1N']
avg_regret = []

for i, algorithm in enumerate(algorithms):
    regret_sum = 0
    for trial in range(Trial):
        algorithm.clear()
        algorithm.run(env)
        regret_sum += algorithm.get_cum_rgt()[-1]
    avg_regret.append(regret_sum / Trial)

plt.figure(figsize=(6, 5))
for i, algorithm in enumerate(algorithms):
    plt.plot(range(T), algorithm.get_cum_rgt(), label=algorithm_names[i])
plt.xlabel('t (Trials)', fontsize=15)
plt.ylabel('Cumulative Regret', fontsize=15)
plt.legend(loc='upper right')
plt.title('Cumulative Regret')
#plt.show()


plt.figure(figsize=(6, 5))
for i, algorithm in enumerate(algorithms):
    plt.plot(range(T), algorithm.get_avg_rgt(), label=algorithm_names[i])
plt.xlabel('t (Trials)', fontsize=15)
plt.ylabel('Average Regret', fontsize=15)
plt.legend(loc='upper right')
plt.title('Average Regret')
plt.show()


Except for our UCB1N implementation, which shows inconsistent behaviour for at least 200 timesteps, all implementations achieve low regret in a short time, within only 100 timesteps. Notably though, UCB1 has trouble converging whithin this frame. UCB2 and Epsilon-Greedy on the other hand manage to almost exclusively pick the best arm only towards the end of the test. Interestingly, while UCB2 is an even more sophisticated version of UCB1, Epsilon-Greedy reaches the same or better level of performance in practise with a much more simplistic approach.

## Test 2
The second test will be conducted for the cooperative UCB strategies. Here, multiple agents will interact with the environment themselves, but share information on the findings with each other. However, not all agents are necessarily pairwise connected. For example, in our test, agent 1 can communicate with agents 2 and 5 (and vice versa).

In [ ]:
# -*- coding: utf-8 -*-
"""
Created on Fri May 31 11:42:42 2024

@author: Xiaotong

test the algorithms in paper "Distributed cooperative decision making in multi-agent multi-armed bandits"
"""

# main.py
import matplotlib.pyplot as plt
from environment.env import env_stochastic

from algorithms.CoopUCB2sl import CoopUCB2Selective
from algorithms.CoopUCB2 import CoopUCB2

# Parameters
T = 5000  # Number of trials
num_agents = 5
num_arms = 10
mu = [0.4, 0.5, 0.5, 0.6, 0.7, 0.7, 0.8, 0.9, 0.92, 0.95]
observation_matrix = [
    [0, 1, 0, 0, 1],
    [1, 0, 1, 0, 0],
    [0, 1, 0, 1, 0],
    [0, 0, 1, 0, 1],
    [1, 0, 0, 1, 0 ],
]
gamma = 1
eta = 0.5
sigma_g = 2.5
# Initialize environment and algorithm

noise = gn(1,0,0.01,[-0.1,0.1])
env = env_stochastic(num_arms, mu, noise)

algorithm = CoopUCB2(T, num_agents, num_arms, observation_matrix, gamma, sigma_g, eta)
algorithm2 = CoopUCB2Selective(T, num_agents, num_arms, observation_matrix, gamma, sigma_g, eta)

# Run the algorithm
algorithm.run(env)
algorithm2.run(env)
# Get average regret
avg_regret = algorithm.get_avg_rgt()
cum_regret = algorithm.get_cum_rgt()

avg_regret2 = algorithm2.get_avg_rgt()
cum_regret2 = algorithm2.get_cum_rgt()


# Plotting
plt.plot(avg_regret)
plt.xlabel('Trials')
plt.ylabel('Average Regret')
plt.title('Performance of coop_ucb Algorithm')
plt.grid(True)
plt.show()

# Plotting
plt.plot(cum_regret)
plt.xlabel('Trials')
plt.ylabel('Cummulative Regret')
plt.title('Performance of coop_ucb Algorithm')
plt.grid(True)
plt.show()

fig, axs = plt.subplots( 2,figsize=(12, 5))
# Plot cumulative regret for each agent for Case 1
for agent in range(num_agents):
    axs[0].plot(algorithm.cumulative_regret[agent], label=f'Agent {agent+1}')
axs[0].set_title('Cumulative Regret of coopUCB Algorithm')
axs[0].set_xlabel('Trials')
axs[0].set_ylabel('Cumulative Regret')
axs[0].legend()
axs[0].grid(True)

for agent in range(num_agents):
    axs[1].plot(algorithm.average_regret[agent], label=f'Agent {agent+1}')
axs[1].set_title('Cumulative Regret of coopUCB Algorithm')
axs[1].set_xlabel('Trials')
axs[1].set_ylabel('Average Regret')
axs[1].legend()
axs[1].grid(True)

plt.tight_layout()
plt.show()

# Plotting
plt.plot(avg_regret2)
plt.xlabel('Trials')
plt.ylabel('Average Regret')
plt.title('Performance of coop_ucb Algorithm')
plt.grid(True)
plt.show()

# Plotting
plt.plot(cum_regret2)
plt.xlabel('Trials')
plt.ylabel('Cummulative Regret')
plt.title('Performance of coop_ucb Algorithm')
plt.grid(True)
plt.show()

fig, axs = plt.subplots( 2,figsize=(12, 5))
# Plot cumulative regret for each agent for Case 1
for agent in range(num_agents):
    axs[0].plot(algorithm2.cumulative_regret[agent], label=f'Agent {agent+1}')
axs[0].set_title('Cumulative Regret of coopUCB Algorithm')
axs[0].set_xlabel('Trials')
axs[0].set_ylabel('Cumulative Regret')
axs[0].legend()
axs[0].grid(True)

for agent in range(num_agents):
    axs[1].plot(algorithm2.average_regret[agent], label=f'Agent {agent+1}')
axs[1].set_title('Average Regret of coopUCB Algorithm')
axs[1].set_xlabel('Trials')
axs[1].set_ylabel('Average Regret')
axs[1].legend()
axs[1].grid(True)

plt.tight_layout()
plt.show()


## Test 3
This case will test our implementation of the MAMAB_SI strategy. Similar to before, it employs multiple UCB-agents, but instead of directly sharing estimates with each other, each agent only has an individual chance of observing its neighbours at a given time and only learns about this instantaneous result then.

In [ ]:
# -*- coding: utf-8 -*-
"""
Created on Fri May 31 11:51:09 2024
test algorithm in "Heterogeneous Stochastic Interactions for Multiple Agents in a Multi-armed Bandit Problem"
@author: Xiaotong
"""

# main.py
import matplotlib.pyplot as plt
from environment.env import env_stochastic

from algorithms.MAMAB_SI import MAMAB_SI

# Parameters
T = 50000  # Number of trials
num_agents = 6
num_arms = 10
mu = [0.4, 0.5, 0.5, 0.6, 0.7, 0.7, 0.8, 0.9, 0.92, 0.95]
sociabilities = [0.50, 0.85, 0.05, 0.50, 1.00, 0.90]
observation_matrix = [
    [0, 1, 1, 1, 1, 1],
    [1, 0, 1, 1, 1, 1],
    [1, 1, 0, 1, 1, 1],
    [1, 1, 1, 0, 1, 1],
    [1, 1, 1, 1, 0, 1],
    [1, 1, 1, 1, 1, 0]
]

# Initialize environment and algorithm
noise = gn(1,0,0.01,[-0.1,0.1])
env = env_stochastic(num_arms, mu, noise)
algorithm = MAMAB_SI(T, num_agents, num_arms, sociabilities, observation_matrix)

# Run the algorithm
algorithm.run(env)

# Get average regret
avg_regret = algorithm.get_avg_rgt()
cum_regret = algorithm.get_cum_rgt()


# Plotting
plt.plot(avg_regret)
plt.xlabel('Trials')
plt.ylabel('Average Regret')
plt.title('Performance of MAMAB_SI Algorithm')
plt.grid(True)
plt.show()

# Plotting
plt.plot(cum_regret)
plt.xlabel('Trials')
plt.ylabel('Average Regret')
plt.title('Performance of MAMAB_SI Algorithm')
plt.grid(True)
plt.show()


We did not test this strategy further than that, so other researchers may find it interesting to look more into the details. For example, the original paper predicts that an agent is most likely to perform well if is has a high sociability, but is connected to agents with a low one, so it profits from the exploration others do and these others actually do this exploration because they hardly look at others' findings themselves.

## Test 4
This case tests yet another multi agent flavour of UCB, the MUCB strategy. As before, it was considered as standalone and not picked up by us again later on.

In [ ]:
# -*- coding: utf-8 -*-
"""
Created on Fri May 31 12:00:06 2024

test algorithm in "Combinatorial Network Optimization With Unknown Variables: Multi-Armed BanditsWith Linear Rewards and Individual Observations"

@author: Xiaotong
"""

import matplotlib.pyplot as plt

from algorithms.mucb import MUCB
from environment.env import env_stochastic

num_agents = 4
num_arms = 6
num_trials = 1000
ucb1 = MUCB(num_agents, num_trials, num_arms)
mu = [0.1, 0.2, 0.3, 0.4, 0.5, 0.8]  

noise = gn(1,0,0.01,[-0.1,0.1])
env = env_stochastic(num_arms, mu, noise)

num_simulations = 1

avg_cumulative_regret = np.zeros(num_trials)
avg_average_regret = np.zeros(num_trials)

for _ in range(num_simulations):
    ucb1 = MUCB(num_agents, num_trials, num_arms)
    ucb1.run(env)
    avg_cumulative_regret += ucb1.get_cum_rgt()
    avg_average_regret += ucb1.get_avg_rgt()
    ucb1.clear()


avg_cumulative_regret /= num_simulations
avg_average_regret /= num_simulations

plt.figure(figsize=(10, 5))
plt.plot(range(num_trials), avg_average_regret)
plt.title('Average Regret Over Time')
plt.xlabel('Time Step')
plt.ylabel('Average Regret')
plt.show()

plt.figure(figsize=(10, 5))
plt.plot(range(num_trials), avg_cumulative_regret)
plt.title('Cumulative Regret Over Time')
plt.xlabel('Time Step')
plt.ylabel('Cumulative Regret')
plt.show()


## Test Modular
This test serves the sole purpose of asserting that our modular implementation of UCB1 does the same as our previous one. We starting modularizing the strategies to avoid redundances in code, partly by utilizing inheritance, and to be able to freely combine strategies with different breakpoint adaption mechanics that only had a specific strategy in mind (usually UCB with forced exploration) in their original papers.

In [ ]:
from environment.env import env_stochastic
from algorithms.UCB1 import UCB1
from algorithms.modular.moduleUsers.ucb import ModularUCB

if __name__ == "__main__":
	num_arm = 5
	T = 1000

	# Stochastic
	mu = np.array([0.1, 0.9, 0.4, 0.3, 0.6])
	Trial = 10
	noise = gn(1,0,0.01,[-0.1,0.1])
	envs = list()
	envs.append(env_stochastic(num_arm, mu, noise))


	algorithms = [UCB1(T, num_arm,2), ModularUCB(T, num_arm,2)]
	algorithm_names = ['Original UCB', 'Modular implementation']
	env_names = ['stochastic']

	tester.test(T, Trial, envs, algorithms, algorithm_names, env_names)


# Miscellaneous
## Test 5
This tests will match UCB1 and exp3 against the usual environment from before, but also against three different adversary environments that would generally allow no assumptions on their nature and may even be designed to confuse specific strategies. A strategy specifically tailored for this scenario is the exp3-strategy.

In [ ]:
# -*- coding: utf-8 -*-
"""
Created on Fri May 31 11:17:30 2024

@author: Xiaotong

test the algorithms in paper "Finite-time Analysis of the Multiarmed Bandit Problem"
"""


from environment.env import env_stochastic
from environment.envAd1 import env_adverse1
from environment.envAd2 import env_adverse2
from environment.envAd3 import env_adverse3
from algorithms.exp3 import exp3
from algorithms.epsilion import EpsilonGreedy
from algorithms.UCB1 import UCB1

num_arm = 5
T = 1000

# Stochastic
mu = np.array([0.1, 0.9, 0.4, 0.3, 0.6])
Trial = 10
noise = gn(1,0,0.01,[-0.1,0.1])
envs = list()
envs.append(env_stochastic(num_arm, mu, noise))
envs.append(env_adverse1(num_arm, noise))
envs.append(env_adverse2(num_arm, noise))
envs.append(env_adverse3(num_arm, noise, difficulty=100))


algorithms = [exp3(T, num_arm, alpha=0.1, gamma=0.3), UCB1(T, num_arm)]
algorithm_names = ['exp3', 'UCB1']
env_names = ['stochastic', 'biggerBetter', 'random', 'shifting']

tester.test(T, Trial, envs, algorithms, algorithm_names, env_names)


As can be expected, UCB1 perform best on the normal, stochastic environment. exp3 is not much worse here, but stays with a good linear regret instead of converging further.

The other settings have failed to highlight the specific advantages of exp3. The environment where arms with higher indices yield better rewards is basically some kind of stochastic environment too, which is why UCB1 has the edge here as well. The environment where the best arm and its vicinity shift around on regular intervals is closer to the non stationary settings in the next section, for which exp3 seems better suited too. As for the random environment, arguably any strategy is optimal because the best arm is decided at random for each timestep, so if there was a strategy that picked the best arm in more than 1/K cases on average, it would be able to predict the random draws that we consider to be truly random.

## Test Mult
This test uses a stochastic environment, but with the MAB where the agent has to choose some subset of arms with a fixed size greater than 1 instead of just one arm. (Here: 3 out of 10 arms each timestep.) These have to different arms and the reward that the agent has to optimize is their combined feedback. We did the test in order to test our implementation of exp3M, a strategy that is suited for such a setting. Like the normal exp3, it assigns weights to the arms, but then extracts multiple (here 3) choices from that instead of picking one arm with the derived probability distribution.

In [ ]:
from environment.env import env_stochastic
from algorithms.exp3M import exp3M

if __name__ == "__main__":
	num_arm = 10
	batch_size = 3
	T = 1000

	# Stochastic
	mu = np.array([0.1, 0.9, 0.4, 0.3, 0.6, 0.1, 0.001, 0.95, 0.7, 0.6])
	Trial = 10
	noise = gn(1,0,0.01,[-0.1,0.1])
	envs = list()
	envs.append(env_stochastic(num_arm, mu, noise))


	algorithms = [exp3M(T, num_arm, batch_size, gamma=0.1)]
	algorithm_names = ['exp3M']
	env_names = ['stochastic']
    #env_name.append('biggerBetter')
    #env_name.append('random')
    #env_name.append('shifting')

	tester.test(T, Trial, envs, algorithms, algorithm_names, env_names)


What may look like a run of the usual exp3 now actually needs to quickly find and exploit the 3 best arms instead of the single best arm. As can be seen, with sufficient success.

## Test Contextual

In [ ]:
from environment.envContextual import EnvContextual
from algorithms.UCB1 import UCB1
from algorithms.modular.moduleUsers.linUCB import LinUCB
from algorithms.modular.moduleUsers.linUCB2 import LinUCB2

if __name__ == '__main__':
	num_arm = 7
	T = 10000

	# Stochastic
	user_features = [0.5, 0.3, 0.4, 0.2, 0.9, 0]
	Trial = 8
	noise = gn(1,0,0.01,[-0.2,0.2])
	envs = list()
	envs.append(EnvContextual(num_arm, user_features, noise, False))
	envs.append(EnvContextual(num_arm, user_features, noise, True))


	algorithms = []
	algorithms.append(LinUCB(T, num_arm, len(user_features), alpha=0.5))
	algorithms.append(LinUCB2(T, num_arm, len(user_features), alpha=0.5))
	algorithms.append(UCB1(T, num_arm, xi=0.5))
    
	algorithm_names = []
	algorithm_names.append("LinUCB")
	algorithm_names.append("UCB1")
	algorithm_names.append("LinUCB variant")
	env_names = ["contextual", "contextual_renewing"]

	print(algorithm_names)
	tester.test(T, Trial, envs, algorithms, algorithm_names, env_names)

# Non Stationary, Single Objective
These tests will compare the performance of different strategies for a MAB with breakpoints (or changepoints), points in time where the expected feedback of one or all arms abruptly changes.

## Test 1
This test matches the standard UCB algorithm against a sliding window version, which forgets past results as soon as they are farther back in time than the window size, and a Discount version, which diminishes all past results by a set factor every timestep.  
They run on a normal environment where the rewards are stable (except for noise) and one with two breakpoints. Note that arm 1 starts with the best reward (0.5) before being overtaken by arm 2 (0.9) and finally returning to the way it was before. Also note that arm 1 is stable despite the breakpoints.

In [ ]:
from environment.env import env_stochastic
from environment.envNonStationary import env_non_stationary
from algorithms.UCB1 import UCB1
from algorithms.discountedUCB import DiscountedUCB
from algorithms.slidingWindowUCB import SlidingWindowUCB

if __name__ == '__main__':
	num_arm = 3
	T = 10000

	# Stochastic
	mu1 = np.array([0.5, 0.3, 0.4])
	mu2 = np.array([0.5, 0.3, 0.9])
	mu3 = np.array([0.5, 0.3, 0.4])
	Trial = 8
	noise = gn(1,0,0.01,[-0.2,0.2])
	envs = list()
	envs.append(env_stochastic(num_arm, mu1, noise))
	envs.append(env_non_stationary(num_arm, [mu1, mu2, mu3], noise, [3000, 5000]))


	algorithms = [UCB1(T, num_arm, xi=0.5), DiscountedUCB(T, num_arm, 0.9975, xi=0.5), SlidingWindowUCB(T, num_arm, 800, xi=0.5)]
	algorithm_names = ["UCB1", "DiscountedUCB", "SlidingWindowUCB"]
	env_names = ["stochastic", "non stationary"]

	print(algorithm_names)
	tester.test(T, Trial, envs, algorithms, algorithm_names, env_names)

As can be seen, as long as the whole environment stays stable, the UCB-strategy performs best and is also the only one to achieve sub-linear regret. The other two strategies fail at this aspect: Because both of them gradually discard their history in some way, they are forced to pull non-optimal arms on a regular basis in order to renew their history, leading to linear regret.  
On the other hand, this is what enables them to react to breakpoints much faster than standard UCB, which keeps pulling arms that are no longer optimal without noticing.  
Note that, while Sliding Window and Discount perform the same asymptotically, Sliding Window is slightly better and its performance is more coherent in both environments.

## Test 2
This test adds forced-exploration-UCB with the breakpoint detector from GLR_klUCB, another with the BOCD breakpoint detector and yet another with the Monitor from Monitored UCB, while dropping Standard UCB and the stable environment, which where already covered in the previous test. Discount is dropped too because it performed almost the same as Sliding window, which has been left in the test.

In [ ]:
from environment.env import env_stochastic
from environment.envNonStationary import env_non_stationary
from algorithms.UCB1 import UCB1
from algorithms.slidingWindowUCB import SlidingWindowUCB
from algorithms.monitoredUCB import MonitoredUCB
from algorithms.modular.moduleUsers.glr_klUCB import GLR_klUCB
from algorithms.modular.moduleUsers.bocd import BOCD

if __name__ == '__main__':
	num_arm = 3
	T = 10000

	mu1 = np.array([0.5, 0.3, 0.4])
	mu2 = np.array([0.5, 0.3, 0.9])
	mu3 = np.array([0.5, 0.3, 0.4])
	Trial = 8
	noise = gn(1,0,0.01,[-0.2,0.2])
	envs = list()
	envs.append(env_non_stationary(num_arm, [mu1, mu2, mu3], noise, [3000, 5000]))

	# tuning for
	# T=10000, M=3, v(i+1)-v(i) >= 2000, delta(k,i) = 0.5
	# So 3 < floor(10000/L) and 2000 > L
	# and 0.5 >= 2sqrt(log(2KT^2)/w) + 2sqrt(log(2T)/w)
	# => 0.5 >= 2(2.881 + 2.074)/sqrt(w) => 0.5*sqrt(w) >= 9.91

	# One solution: w = 50, L=50*ceil(3/gamma)
	# gamma=0.1
	#406,0.65

	algorithms = list()
	algorithms.append(SlidingWindowUCB(T, num_arm, 800, xi=0.5))
	algorithms.append(MonitoredUCB(T, num_arm, w=50, b=3, gamma=0.1))
	algorithms.append(GLR_klUCB(T, num_arm, alpha=0.03, delta=0.01, global_restart=True, lazyness=10))
	algorithms.append(GLR_klUCB(T, num_arm, alpha=0.03, delta=0.01, global_restart=False, lazyness=10))
	algorithms.append(BOCD(T, num_arm, alpha=0.03))

	algorithm_names = ["SlidingWindowUCB", "MonitoredUCB", "GLR-klUCB glob", "GLR-klUCB loc", "BOCD"]
	env_names = ["non stationary"]

	tester.test(T, Trial, envs, algorithms, algorithm_names, env_names)


Again, we get coherent performance from the passive breakpoint adaptor, the Sliding Window, though this time, it is eventually outperformed by the active adaptors when there are no more breakpoints.  
The Monitor outperforms the more sophisticated GLR detector, but arguably because the breakpoints here are easy to notice. Due to GLR testing all possible window segmentations, it may be more reliably in settings not covered here.  
Speaking of GLR, the local reset version (that only resets the arm in which the breakpoint was detected) notably outperforms the global reset version (that resets everything). However, this comes with a high cost: The global version is already about 10 times slower than the other adaptors and the locals version is 2 times than the global still, and this is already when skipping 9 out of 10 calls (lazyness). The local version is slower because, when less data is reset, there naturally is more data left to compute on.  
Asserting the perfomance of BOCD is less straight forward. It exhibits an extensive variation and thus poor reliability in one-shot experiments, and it is neither the best-performing nor the most time-efficient approach. On the other hand, works without any parameters at all without being significantly worse than adaptors that require much more fine tuning for the setting, the details of which might not be known beforehand.

## Monitor efficency test
This test is to compare 2 versions of Monitored UCB: One that faithfully implements the original algorithm and sums up all numbers in the 2 windows every time a check takes place, and another that saves the sum and only adds the new and substract the new value, cutting time consumption for the check from O(w) to O(1), at the cost of possibly losing accuracy due to limited floating point precision.

In [ ]:
from environment.envNonStationary import env_non_stationary
from algorithms.monitoredUCB import MonitoredUCB
from algorithms.monitoredUCBOriginal import MonitoredUCBOriginal

if __name__ == "__main__":
	num_arm = 3
	T = 10000

	mu1 = np.array([0.5, 0.3, 0.4])
	mu2 = np.array([0.5, 0.3, 0.9])
	mu3 = np.array([0.5, 0.3, 0.4])
	Trial = 4
	noise = gn(1,0,0.01,[-0.2,0.2])
	envs = list()
	envs.append(env_non_stationary(num_arm, [mu1, mu2, mu3], noise, [3000, 5000]))

	algorithms = [MonitoredUCB(T, num_arm, w=50, b=3, gamma=0.1), MonitoredUCBOriginal(T, num_arm, w=50, b=3, gamma=0.1)]
	algorithm_names = ["MonitoredUCB", "MonitoredUCBOriginal"]
	env_names = ["non stationary"]

	tester.test(T, Trial, envs, algorithms, algorithm_names, env_names)


As can be seen, the simplified version basically performs the same, but requires only about 3/4 of the time. This means there is no reason not to use it, which was already done in test 1 and test 2.

## Modular Monitor verification
Our first implementation of Monitored UCB monolithically implemented the whole strategy, but we switched to modular approaches later on. This test is simply to ensure there was no mistake in the transition.

In [ ]:
from environment.envNonStationary import env_non_stationary
from algorithms.monitoredUCB import MonitoredUCB
from algorithms.modular.moduleUsers.monitoredUCB import MonitoredUCB as MonitoredUCBModular

if __name__ == "__main__":
	num_arm = 3
	T = 10000

	mu1 = np.array([0.5, 0.3, 0.4])
	mu2 = np.array([0.5, 0.3, 0.9])
	mu3 = np.array([0.5, 0.3, 0.4])
	Trial = 10
	noise = gn(1,0,0.01,[-0.2,0.2])
	envs = list()
	envs.append(env_non_stationary(num_arm, [mu1, mu2, mu3], noise, [3000, 5000]))


	algorithms = list()
	algorithms.append(MonitoredUCB(T, num_arm, w=50, b=3, gamma=0.1))
	algorithms.append(MonitoredUCBModular(T, num_arm, w=50, b=3, gamma=0.1))

	algorithm_names = ["Old", "Modular"]
	env_names = ["non stationary"]

	tester.test(T, Trial, envs, algorithms, algorithm_names, env_names)
	# Basically the same results, but:
	#Average time for Old: 1.102609669999947 seconds.
	#Average time for Modular: 1.1457539999999427 seconds.

	# Open issue, low priority: Why is the modular version notably slower? (Object-orientation overhead or something avoidable?)


# Continuous Optimization
The following tests test our implementations for strategies for continuous optimization, that is, a possibly unknown, possibly changing function that accepts a multi-dimensional input and returns a cost that we want to minimize on the long term.

## Test 1
The first test compares the performance of Greedy Projection and of BGD on 3 different settings: One where the cost function stays the same, one where it sometimes shifts to another cost function (with the optimal points being distributed around a non-changin center) and one where it shifts after every timestep.

In [ ]:
from environment.envParabola import EnvParabola
from algorithms.greedyProjection import GreedyProjection
from algorithms.bgd import BGD


if __name__ == "__main__":
	T = 100
	repeats = 10

	envs = list()
	envs.append(EnvParabola(dimensions=3, pos_mean=0, pos_scale=5, slope_mean=1, slope_scale=0.5, boundaries=10, stability=1))
	envs.append(EnvParabola(dimensions=3, pos_mean=0, pos_scale=5, slope_mean=1, slope_scale=0.5, boundaries=10, stability=0.3))
	envs.append(EnvParabola(dimensions=3, pos_mean=0, pos_scale=5, slope_mean=1, slope_scale=0.5, boundaries=10, stability=0))


	algorithms = [GreedyProjection(T), BGD(T)]
	algorithm_names = ["Greedy Projection", "BGD"]
	env_names = ["fully stable", "less stable", "unstable"]

	tester.test(T, repeats, envs, algorithms, algorithm_names, env_names)


The fully stable environment obviously is the optimal setting for Greedy Projection: It quickly gets close to the optimal costs and then only needs to stay there. Note that in all other cases, BGD can keep up with it, which is impressive because it never sees the cost functions and only operates on the feedback. In the unstable setting, it even manages to outperform Greedy Projection.  
Interestingly, in the less stable setting, Greedy Projection still performs best, though not by much.

## Test 2
In the second test, instability is deterministic: A stability of 0.99 (shift in 0.01 of cases) means that a shift occurs after _exactly_ every 100 timesteps.  
This time, we match up the previous BGD against its experts version.

In [ ]:
from environment.envParabola import EnvParabola
from algorithms.bgd import BGD
from algorithms.modular.metaPBGD import MetaPBGD


if __name__ == "__main__":
	T = 1000
	repeats = 10

	envs = []
	envs.append(EnvParabola(dimensions=3, pos_mean=0, pos_scale=5, slope_mean=1, slope_scale=0.5, boundaries=10, stability=0.99, fixed_breakpoints=True))
	envs.append(EnvParabola(dimensions=3, pos_mean=0, pos_scale=5, slope_mean=1, slope_scale=0.5, boundaries=10, stability=1))


	algorithms = []
	algorithms.append(MetaPBGD(T))
	algorithms.append(BGD(T))
	algorithm_names = ["MetaBGD", "BGD"]
	env_names = ["almost stable", "fully stable"]

	tester.test(T, repeats, envs, algorithms, algorithm_names, env_names)

## LTC Test
The last test tests on environment with not only boundaries, but also constraints. These constraints can be violated, but should be kept on average. TODO: Fix the BGD version.

In [ ]:
from environment.envParabola import EnvParabola
from algorithms.bgd import BGD
from algorithms.gb_oco_ltc import GB_OCO_LTC
from algorithms.gb_oco_ltc_bgd import GB_OCO_LTC_BGD
#from algorithms.modular.metaPBGD import MetaPBGD


if __name__ == "__main__":
	T = 10000
	repeats = 8
	
	constrainted_env = EnvParabola(dimensions=3, pos_mean=0.5, pos_scale=0.2, slope_mean=1, slope_scale=0, boundaries=10, stability=1)
	vars = constrainted_env.getVariables()
	constraint1 = vars[0] + vars[1] + vars[2] - 1
	constrainted_env.addConstraint(constraint1)
	
	envs = []
	envs.append(constrainted_env)
	
	
	algorithms = []
	#algorithms.append(MetaPBGD(T))
	algorithms.append(GB_OCO_LTC(T, 0.05, 1))
	algorithms.append(GB_OCO_LTC_BGD(T, 0.05, 1))
	algorithms.append(GB_OCO_LTC_BGD(T, 1000, 1))
	algorithm_names = ["Analytical", "Estimated", "Estimated_constraint_overdrive"]
	env_names = ["env"]
	
	tester.test(T, repeats, envs, algorithms, algorithm_names, env_names)

# Multi Objective
The following tests return to the MAB, but with multi-objective, that is, the costs are multi-dimensional and each dimensions represents a different objective that shall not be ignored. Most strategies consider the Gini Regret, but we also included the Nash Regret.

## Basic Test 1
This only tests the MO OGDE strategy on a multi-objective environment.

In [ ]:
from environment.Gaussian_noise import Gaussian_noise as gn
from environment.multiOutput import EnvMultiOutput
from algorithms.modular.moduleUsers.basicMultiObjective import BasicMultiObjective


if __name__ == "__main__":
	num_arm = 3
	T = 200

	weights = [1, 1/2]

	mu = []
	mu.append(np.array([0.1, 0.3]))
	mu.append(np.array([0.2, 0.1]))
	mu.append(np.array([0.1, 0.4]))

	trial = 10
	noise = gn(1,0,0.01,[-0.2,0.2])
	envs = list()
	envs.append(EnvMultiOutput(num_arm, mu, noise, weights))

	algorithms = list()
	algorithms.append(BasicMultiObjective(T, num_arm, num_objectives=2, delta=0.95, gini_weights=weights))

	algorithm_names = ["MO_OGDE"]
	env_names = ["Simple"]

	tester.test(T, trial, envs, algorithms, algorithm_names, env_names)

## Test Original 1
This tests tries to replicate the results from the original MO-OGDE paper to assert that our implementation works as intended. It will run the strategy on ~100~ 64 different, randomly generated environments for ~10^5~ (10^4)*5 timesteps. (The environments are deepcopies from one root environment, so we set refresh_first to true to perform a re-init after the deepcopy so the randomness is really there.)

In [ ]:
from environment.multiOutputRandomized import EnvMultiOutputRandomized
from algorithms.modular.moduleUsers.basicMultiObjective import BasicMultiObjective

if __name__ == "__main__":
	num_arm = 5
	num_objectives=5

	T = (10**4)*5

	weights = []
	for i in range(0, 20):
		weights.append(1/(2**i))

	trial = 64
	envs = list()
	envs.append(EnvMultiOutputRandomized(num_arm, num_objectives, weights))

	algorithms = list()
	algorithms.append(BasicMultiObjective(T, num_arm=num_arm, num_objectives=num_objectives, delta=0.05, gini_weights=weights))

	algorithm_names = ["MO_OGDE"]
	env_names = ["random environment"]

	tester.test(T, trial, envs, algorithms, algorithm_names, env_names, logscale=True, refresh_first=True)


## Test Original 2
This test is similar to the one above, but this time, its objective is to look whether dropping a learning rate aspect of the strategy that is required for theoretical guarantees has any impact when running it.

Additionally, we further reduced the complexity of the test to make it finish within a more reasonable amount of time.

In [ ]:
from environment.multiOutputRandomized import EnvMultiOutputRandomized
from algorithms.modular.moduleUsers.basicMultiObjective import BasicMultiObjective
from algorithms.modular.moduleUsers.deltalessMultiObjective import DeltalessMultiObjective

if __name__ == "__main__":
	num_arm = 5
	num_objectives=5

	T = 10**4

	weights = []
	for i in range(0, 20):
		weights.append(1/(2**i))

	trial = 64
	envs = list()
	envs.append(EnvMultiOutputRandomized(num_arm, num_objectives, weights))

	algorithms = list()
	algorithms.append(DeltalessMultiObjective(T, num_arm=num_arm, num_objectives=num_objectives, gini_weights=weights))
	algorithms.append(BasicMultiObjective(T, num_arm=num_arm, num_objectives=num_objectives, delta=0.05, gini_weights=weights))

	algorithm_names = ["Deltaless MO_OGDE", "MO_OGDE"]
	env_names = ["random environment"]

	tester.test(T, trial, envs, algorithms, algorithm_names, env_names, logscale=True, refresh_first=True)


The deltaless version is at the very least on par with the original version, at least from a practical point of view. So in the end, we did not break the strategy, which allowed us to further follow this approach.

## Test Original 3
Previously, we managed to free the strategy from its special learning rate parameter without measurable impact on performance. This allowed us in the next step to instead include a parameter that directly controls the learning rate and build a meta learning strategy with experts from that. This test is to see the differences in performance in the original testcase.

In [ ]:
from environment.multiOutputRandomized import EnvMultiOutputRandomized
from algorithms.modular.moduleUsers.basicMultiObjective import BasicMultiObjective
from algorithms.modular.moduleUsers.expertsMultiObjective import ExpertsMultiObjective

if __name__ == "__main__":
	num_arm = 5
	num_objectives=5

	T = 10**4

	weights = []
	for i in range(0, 20):
		weights.append(1/(2**i))

	trial = 16
	envs = list()
	envs.append(EnvMultiOutputRandomized(num_arm, num_objectives, weights))

	algorithms = list()
	algorithms.append(ExpertsMultiObjective(T, num_arm=num_arm, num_objectives=num_objectives, gini_weights=weights))
	algorithms.append(BasicMultiObjective(T, num_arm=num_arm, num_objectives=num_objectives, delta=0.05, gini_weights=weights))

	algorithm_names = ["Meta MO_OGDE", "MO_OGDE"]
	env_names = ["random environment"]

	tester.test(T, trial, envs, algorithms, algorithm_names, env_names, logscale=True, refresh_first=True)


By appliying the experts paradigm to the strategy, we managed to improve its performance by a little, at least in practise and for some cases, without worsening it in others. Do note however that this comes wih a considerable impact on the runtime. As long as the environment is stationary, there is no real benefit, but we test the strategy for non-stationary settings later on.

## Breakpoint Test 1
This test set runs on an environment that is similar to one in basic test 1, but includes breakpoints. As a novel element, the MO OGDE strategy has been combined with some of the breakpoint adaptors originally intended for usage in single objective settings.

In [ ]:
from environment.multiOutput import EnvMultiOutput
from environment.multiOutputNonStationary import EnvMultiOutputNonStationary
from algorithms.modular.moduleUsers.basicMultiObjective import BasicMultiObjective
from algorithms.modular.moduleUsers.slidingWindowMO import SlidingWindowMO
from algorithms.modular.moduleUsers.discountedMO import DiscountedMO
from algorithms.modular.moduleUsers.monitoredMO import MonitoredMO

if __name__ == "__main__":
	num_arm = 3
	T = 200

	weights = [1, 1/2]

	mu1 = []
	mu1.append(np.array([0.1, 0.3]))
	mu1.append(np.array([0.2, 0.1]))
	mu1.append(np.array([0.1, 0.4]))

	"""mu2 = []
	mu2.append(np.array([0.12, 0.17]))
	mu2.append(np.array([0.15, 0.2]))
	mu2.append(np.array([0.1, 0.4]))"""

	mu2 = []
	mu2.append(np.array([0.1, 0.3]))
	mu2.append(np.array([0.1, 0.4]))
	mu2.append(np.array([0.2, 0.1]))

	mu3 = []
	mu3.append(np.array([0.1, 0.3]))
	mu3.append(np.array([0.2, 0.1]))
	mu3.append(np.array([0.1, 0.4]))

	trial = 10
	noise = gn(1,0,0.01,[-0.2,0.2])
	envs = list()
	envs.append(EnvMultiOutput(num_arm, mu1, noise, weights))
	envs.append(EnvMultiOutputNonStationary(num_arm, [mu1, mu2, mu3], noise, weights, [100, 150]))

	algorithms = list()
	algorithms.append(BasicMultiObjective(T, num_arm, num_objectives=2, delta=0.05, gini_weights=weights))
	algorithms.append(SlidingWindowMO(T, num_arm, num_objectives=2, delta=0.05, gini_weights=weights, window_len=20))
	algorithms.append(DiscountedMO(T, num_arm, num_objectives=2, delta=0.05, gini_weights=weights, gamma=0.95))
	algorithms.append(MonitoredMO(T, num_arm, num_objectives=2, delta=0.05, gini_weights=weights, w=30, b=1))

	algorithm_names = []
	algorithm_names.append("MO_OGDE")
	algorithm_names.append("SlidingWindow")
	algorithm_names.append("Discounted")
	algorithm_names.append("Monitored")
	env_names = []
	env_names.append("Simple")
	env_names.append("withBreakpoints")

	tester.test(T, trial, envs, algorithms, algorithm_names, env_names)

The main takeaway is that the original MO OGDE outperforms the other strategies in the breakpoint-less environment (which is no surprise), but is also the only one to be obviously thrown off by breakpoints in the other environment. (The result presentation will be improved in the next test.)

## Breakpoint Test 2
In this test, we greatly increase the number of timesteps and the number of repetitions while dropping the breakpoint-less environment, which is not relevant at this point and would overly clutter the result visualization. Instead, we include two more adaptors.

In [ ]:
from environment.multiOutput import EnvMultiOutput
from environment.multiOutputNonStationary import EnvMultiOutputNonStationary
from algorithms.modular.moduleUsers.basicMultiObjective import BasicMultiObjective
from algorithms.modular.moduleUsers.slidingWindowMO import SlidingWindowMO
from algorithms.modular.moduleUsers.discountedMO import DiscountedMO
from algorithms.modular.moduleUsers.monitoredMO import MonitoredMO
from algorithms.modular.moduleUsers.bocdMO import BOCD_MO
from algorithms.modular.moduleUsers.glrMO import GLR_MO

if __name__ == "__main__":
	num_arm = 3
	T = 10000

	weights = [1, 1/2]

	mu1 = []
	mu1.append(np.array([0.1, 0.3]))
	mu1.append(np.array([0.2, 0.1]))
	mu1.append(np.array([0.1, 0.4]))

	mu2 = []
	mu2.append(np.array([0.1, 0.3]))
	mu2.append(np.array([0.1, 0.4]))
	mu2.append(np.array([0.2, 0.1]))

	mu3 = []
	mu3.append(np.array([0.1, 0.3]))
	mu3.append(np.array([0.2, 0.1]))
	mu3.append(np.array([0.1, 0.4]))

	trial = 16
	noise = gn(1,0,0.01,[-0.2,0.2])
	envs = list()
	envs.append(EnvMultiOutputNonStationary(num_arm, [mu1, mu2, mu3], noise, weights, [3000, 5000]))

	delta = 0.05
	algorithms = list()
	algorithms.append(BasicMultiObjective(T, num_arm, num_objectives=2, delta=delta, gini_weights=weights))
	algorithms.append(SlidingWindowMO(T, num_arm, num_objectives=2, delta=delta, gini_weights=weights, window_len=800))
	algorithms.append(DiscountedMO(T, num_arm, num_objectives=2, delta=delta, gini_weights=weights, gamma=0.9975))
	algorithms.append(MonitoredMO(T, num_arm, num_objectives=2, delta=delta, gini_weights=weights, w=50, b=3))
	algorithms.append(BOCD_MO(T, num_arm, num_objectives=2, delta=delta, gini_weights=weights))
	algorithms.append(GLR_MO(T, num_arm, num_objectives=2, delta=delta, delta2=0.01, global_restart=True, lazyness=10, gini_weights=weights))

	algorithm_names = []
	algorithm_names.append("MO_OGDE")
	algorithm_names.append("SlidingWindow")
	algorithm_names.append("Discounted")
	algorithm_names.append("Monitored")
	algorithm_names.append("BOCD_MO")
	algorithm_names.append("GLR_MO")
	env_names = []
	env_names.append("withBreakpoints")

	tester.test(T, trial, envs, algorithms, algorithm_names, env_names)


The results are clearer in this version. As before, original MO OGDE cannot handle the breakpoints correctly and loses against all its versions that include a breakpoint adaptor.  
Like in the single objective version, the Monitor adaptor proves to be highly efficient as long as both the window size and the detection threshold are chosen well.  
GLR is on par with the passive adaptors, which might not be enough to justify its runtime overhead, but again, the parameters might not be optimal. Speaking of parameters, BOCD can keep up with the others without any parameters, as before. Interstingly, it seems to perform better in terms of Nash regret and Pareto regret, even though the underlying MO OGDE focuses on the Gini regret.

## Pareto Test
This test compares the performance of Pareto UCB against that of MO OGDE. Note that the former does not optimize for any regret, but only aims to choose arms from the Pareto front, which may or may not still lead to low regret.  
The test runs on two environment, one of which includes bad arms that are still within the Pareto front and thus should be tricky for handle for Pareto UCB.

In [ ]:
from environment.multiOutput import EnvMultiOutput
from algorithms.modular.moduleUsers.paretoUCB import ParetoUCB
from algorithms.modular.moduleUsers.basicMultiObjective import BasicMultiObjective

if __name__ == "__main__":
	num_arm = 4
	T = 3000

	weights = [1, 1/2]

	mu1 = []
	mu1.append(np.array([0.1, 0.3]))
	mu1.append(np.array([0.2, 0.1]))
	mu1.append(np.array([0.15, 0.4]))
	mu1.append(np.array([0.3, 0.2]))
	
	# This one might be more tricky for pareto based strategies because they cannot drop any arm.
	mu2 = []
	mu2.append(np.array([0.1, 0.3]))
	mu2.append(np.array([0.2, 0.1]))
	mu2.append(np.array([0.08, 0.4]))
	mu2.append(np.array([0.3, 0.08]))

	trial = 4
	noise = gn(1,0,0.01,[-0.2,0.2])
	envs = list()
	envs.append(EnvMultiOutput(num_arm, mu1, noise, weights))
	envs.append(EnvMultiOutput(num_arm, mu2, noise, weights))

	delta = 0.05
	algorithms = list()
	algorithms.append(ParetoUCB(T, num_arm, num_objectives=2, alpha=1, gini_weights=weights))
	algorithms.append(BasicMultiObjective(T, num_arm, num_objectives=2, delta=0.05, gini_weights=weights))

	algorithm_names = []
	algorithm_names.append("ParetoUCB")
	algorithm_names.append("MO_OGDE")
	env_names = []
	env_names.append("favourable env")
	env_names.append("tricky env")

	tester.test(T, trial, envs, algorithms, algorithm_names, env_names)

As expected, Pareto UCB clearly fails on the tricky environment. It  has to choose random arms from the full set of arms because it cannot drop any arms that are within the Pareto front. On the more favourable environment, it can roughly keep up with MO OGDE, but is still clearly outmatched by it. Except for runtime advantages, Pareto UCB is not recommendable for environments with many objectives.  
(Remember that an arm is not in the Pareto front iff another arm is at least as good in *all* dimensions and better in one, which naturally becomes more unlikely the more dimensions there are.)

## Basic Test 2
This test compares the experts version of MO OGDE against the Fair MO UCB strategy.

In [ ]:
from environment.multiOutput import EnvMultiOutput
from algorithms.modular.moduleUsers.expertsMultiObjective import ExpertsMultiObjective
from algorithms.modular.moduleUsers.fair_MO_UCB import Fair_MO_UCB


if __name__ == "__main__":
	num_arm = 4
	num_objectives=5
	T = 10000
	
	
	mu = []
	mu.append(np.array([.15, .3, .5, .72, .8]))
	mu.append(np.array([.2, .18, .48, .7, .2]))
	mu.append(np.array([.15, .4, .3, .8, .4]))
	mu.append(np.array([.2, .2, .8, .3, .5]))
	
	weights = []
	for i in range(0, num_objectives):
		weights.append(1/(2**i))
	
	noise = gn(1,0,0.01,[-0.2,0.2])
	trial = 8
	envs = list()
	envs.append(EnvMultiOutput(num_arm, mu, noise, weights))

	algorithms = []
	algorithms.append(ExpertsMultiObjective(T, num_arm=num_arm, num_objectives=num_objectives, gini_weights=weights))
	algorithms.append(Fair_MO_UCB(T, num_arm=num_arm, num_objectives=num_objectives, delta=0.1, gini_weights=weights))
	
	algorithm_names = []
	algorithm_names.append("ExpertsMultiObjective")
	algorithm_names.append("Fair MO UCB")
	env_names = ["Normal Environment"]
	
	tester.test(T, trial, envs, algorithms, algorithm_names, env_names)

Fair MO UCB is clearly outmatched in all apects except time efficiency, which is why we no longer consider it.

## Test Meta Breakpoints
Compare the normal meta strategy against ones that have breakpoint adaption against the meta strategy plus breakpoint adaption. We expected the latter to be the best of both worlds in a non-stationary setting.

For the breakpoint adaptors, we choose Discount, which is a well performing passive adaptor, and BOCD, which is an active adaptor that can almost keep up with other active adaptors such as monitor, but conveniently requires no tuning, as mentioned in a previous test.

You may wonder why several function from tester are called instead of test. This is equivalent and to show how the process can be controlled more granulary. For example, by omitting purgeResults and some of the tests, you can plot new tests together with the ones you removed without calculating the latter again.

In [ ]:
from environment.multiOutputNonStationary import EnvMultiOutputNonStationary
from algorithms.modular.moduleUsers.basicMultiObjective import BasicMultiObjective
from algorithms.modular.moduleUsers.discountedMO import DiscountedMO
from algorithms.modular.moduleUsers.monitoredMO import MonitoredMO
from algorithms.modular.moduleUsers.bocdMO import BOCD_MO
from algorithms.modular.moduleUsers.expertsMultiObjective import ExpertsMultiObjective
from algorithms.modular.moduleUsers.discountedExpertsMultiObjective import DiscountedExpertsMultiObjective
from algorithms.modular.moduleUsers.bogdExpertsMultiObjective import BOCD_ExpertsMultiObjective


# Note that this environment is a little mean, especially in the second half.
if __name__ == "__main__":
	num_arm = 3
	num_objectives = 3
	T = 3000

	weights = [1, 1/2, 1/4]

	mu1 = []
	mu1.append(np.array([0.1, 0.3, 0.2]))
	mu1.append(np.array([0.2, 0.1, 0.2]))
	mu1.append(np.array([0.1, 0.4, 0.3]))

	mu2 = []
	mu2.append(np.array([0.1, 0.3, 0.1]))
	mu2.append(np.array([0.1, 0.4, 0.2]))
	mu2.append(np.array([0.2, 0.1, 0.1]))

	mu3 = []
	mu3.append(np.array([0.2, 0.1, 0.2]))
	mu3.append(np.array([0.1, 0.4, 0.3]))
	mu3.append(np.array([0.1, 0.3, 0.2]))

	mu4 = []
	mu4.append(np.array([0.1, 0.3, 0.2]))
	mu4.append(np.array([0.2, 0.1, 0.2]))
	mu4.append(np.array([0.1, 0.4, 0.3]))

	mu5 = []
	mu5.append(np.array([0.1, 0.3, 0.2]))
	mu5.append(np.array([0.2, 0.1, 0.2]))
	mu5.append(np.array([0.1, 0.4, 0.3]))

	trial = 16
	noise = gn(1,0,0.01,[-0.2,0.2])
	envs = list()
	envs.append(EnvMultiOutputNonStationary(num_arm, [mu1, mu2, mu3, mu4, mu5], noise, weights, [700, 1500, 1700, 2300]))

	delta = 0.05
	algorithms = list()
	algorithms.append(DiscountedMO(T, num_arm, num_objectives=num_objectives, delta=delta, gini_weights=weights, gamma=0.99))
	#algorithms.append(MonitoredMO(T, num_arm, num_objectives=num_objectives, delta=delta, gini_weights=weights, w=50, b=3))
	algorithms.append(BOCD_MO(T, num_arm, num_objectives=num_objectives, delta=delta, gini_weights=weights))
	algorithms.append(BasicMultiObjective(T, num_arm=num_arm, num_objectives=num_objectives, delta=0.05, gini_weights=weights))
	algorithms.append(ExpertsMultiObjective(T, num_arm=num_arm, num_objectives=num_objectives, gini_weights=weights))
	algorithms.append(DiscountedExpertsMultiObjective(T, num_arm=num_arm, num_objectives=num_objectives, gini_weights=weights, gamma=0.99))
	algorithms.append(BOCD_ExpertsMultiObjective(T, num_arm=num_arm, num_objectives=num_objectives, gini_weights=weights))

	algorithm_names = []
	algorithm_names.append("Discounted")
	#algorithm_names.append("Monitored")
	algorithm_names.append("BOCD")
	algorithm_names.append("Original")
	algorithm_names.append("Meta")
	algorithm_names.append("Discounted_Meta")
	algorithm_names.append("BOGD_Meta")
	env_names = []
	env_names.append("Breakpoint env")
	
	tester.resultpath = "metaBreakpointsResults/"
	
	tester.purgeResults()
	tester.testOnly(T, trial, envs, algorithms, algorithm_names, env_names)
	tester.readAllResults()
	tester.plotResults()


First we regard the strategies without any breakpoint adaptors, which are the original MO-UCB and the Meta-MO-UCB. We observe two very interesting results: On the one hand, both of them take about equally heavy hits to their performance from the breakpoints. However, the meta version shows much better recovery from the second breakpoint, which becomes most obvious from around timestep 2000 onwards. Obviously, having multiple experts with different learning rates and being able to dynamically shift to the best performing one indeed works out as promised.

Our other focus was whether or not using the meta version instead of the original version further improves the performance of anything that includes a breakpoint adaptor. (Note how any breakpoint adaption makes a strategy outperform the two strategies without, by a large margin.) Here, the results are less unidirectional and pairing the meta strategy together with the BOCD breakpoint adaptor actually worsened performance by a little. A *possible* explanation is the active adaptor resets *all* experts when it believes there has been a breakpoint, dropping the benefit of having multiple experts that can build vastly different opinions over time.

Finally however, when paring the meta version with the Discount breakpoint adaptor, we see the results we expected. Not only does it perform much better than the original version with the Discount, it is on pair or even better (depending on what regret definitions you consider) than both BOCD versions - which can be called impressive for a strategy that only uses passive breakpoint adaption.

# Operating on real data
In this section, we take an sample from the last.fm database and transform it into a MO-MAB-instance with choosable arms and multi-dimensional rewards.

## Processing the data
Please execute readFM.py to generate the bandit data from a last.fm file. It will grab the most active users and the most popular tracks and generate the expected costs per track (arm) so that they are lower the more a user (dimension / objective) has listened to it. Optionally, set use_correction to true to also consider whether a user is even interested in the popular tracks, which will lead to more realistic responses but generally higher costs that do not differ as much.

Four files will be generated: One that considers the full dataset and three that have it arbitrarily divided into daytimes (but still consider the same users and tracks as in the full set) to simulate breakpoints.

Finally, note that this approach simulates an online learning problem from offline data to test how a strategy might cope with live data that gradually comes in, which may not necessarily be what you want.

## Test FM
This test will read in the bandit settings generated in the previous step. It will generate three environments: One where the feedback is the exact costs, one where it is either 0 or 1, but with the probality of the costs, and one that uses the dayphase file instead of the general file to generate a non-stationary setting.

All these problems can be imagined as having to generate a single playlist for all users that shall maximize the general satisfaction and that you can modify on the fly. A real world scenario where this applies would be making some good selection of tracks for a radio broadcast.

In [ ]:
from environment.multiOutput import EnvMultiOutput
from environment.multiOutputBernulli import EnvMultiOutputBernulli
from environment.multiOutputNonStationary import EnvMultiOutputNonStationary
from algorithms.modular.moduleUsers.basicMultiObjective import BasicMultiObjective
from algorithms.modular.moduleUsers.expertsMultiObjective import ExpertsMultiObjective
from algorithms.modular.moduleUsers.paretoUCB import ParetoUCB

if __name__ == "__main__":
	mu = [0]*4
	calls = [0]*4
	for dayphase in range(4):
		if dayphase == 0:
			filename = "../fm_banditified.csv"
		elif dayphase == 1:
			filename = "../fm_banditified_day.csv"
		elif dayphase == 2:
			filename = "../fm_banditified_evening.csv"
		else:
			filename = "../fm_banditified_night.csv"
		
		mu[dayphase] = np.loadtxt(filename, delimiter=',', encoding="utf-8")
		topstring = ""
		with open(filename, encoding="utf-8") as file:
			for line in file:
				if line[0] == '#':
					topstring += line[1:]
					if "calls=" in line:
						calls[dayphase] = int(line.strip().split('=')[1])
				else:
					# We expect the comment at the top, and only at the top.
					break
		print(topstring)
		#print(mu[dayphase])
	assert calls[0] == sum(calls[1:]), "calls in dayphases do not sum up to "+str(calls[0])
	
	num_arm = len(mu[0])
	num_objectives = len(mu[0][0])
	T = 3000
	
	factor = T/calls[0]
	breakpoint1 = round((calls[1] + 1)*factor)
	breakpoint2 = round((calls[1] + calls[2]+1)*factor)
	#print("Breakpoints:", breakpoint1, breakpoint2)
	
	weights = [0]*num_objectives
	for i in range(num_objectives):
		weights[i] = (1/2)**i


	trial = 4
	noise = gn(1,0,0.01,[-0.2,0.2])
	envs = list()
	envs.append(EnvMultiOutput(num_arm, mu[0], noise, weights))
	envs.append(EnvMultiOutputBernulli(num_arm, mu[0], weights))
	envs.append(EnvMultiOutputNonStationary(num_arm, mu[1:], noise, weights, [breakpoint1, breakpoint2]))

	delta = 0.05
	algorithms = list()
	algorithms.append(ParetoUCB(T, num_arm, num_objectives=num_objectives, alpha=1, gini_weights=weights))
	#algorithms.append(BasicMultiObjective(T, num_arm, num_objectives=num_objectives, delta=0.05, gini_weights=weights))
	algorithms.append(ExpertsMultiObjective(T, num_arm, num_objectives=num_objectives, gini_weights=weights))

	algorithm_names = []
	algorithm_names.append("ParetoUCB")
	#algorithm_names.append("MO_OGDE")
	algorithm_names.append("Meta_MO_OGDE")
	env_names = []
	env_names.append("transparent_env")
	env_names.append("bernoulli_env")
	env_names.append("transparent_dayphase_env")

	tester.test(T, trial, envs, algorithms, algorithm_names, env_names)


As could be expected, both strategies struggle more on the dayphase setting. However, they already do so even before the first breakpoint. A possible explanation would be that the set of popular tracks includes some tracks that were not popular at all during certain dayphases, making exploration (every strategy has to perform some exploration early on) much more punishing than in the full setting. Aside from that, ParetoUCB is more successfull with breakpoint handling even without including some explicit breakpoint adaption.

Another surprise is that the strategies show little difference when comparing the transparent version with the bernoulli version. In other words, high precision data hardly has any benefit over binary data. And the latter might be easier measured in the real world, for example by simply looking whether the user skips a suggested track at a given time or chooses to listen to it and directly plugging this information into the strategy without any further processing.